# Pasting of Objects onto X-ray scans

## Comment this out and run others first before running this

In [ ]:
generate_images()

## ***User to define Parameters here

In [ ]:
# Max number of times the algo will attempt to place objects on a single background image. To prevent infinite loops
MAX_ATTEMPTS = 100

# Customisable parameters [min, max]
rotation_range = [0, 360]
number_objects_per_image = [1, 5]
max_magnification = 1.3 
bg_removal_threshold = 250

# Only put on white space?
# Sometimes it may be good to overlay object in non-whitespace region to make training set more robust.
white_space_hunter = True

# Give a name for object to place.
# Type "multiple" if there are multiple classes to place. Class name extracted up to "_"
# So myObject_1.jpg, myObject_2.jpg will be given class "myObject"
obj_class = "my_class"

In [ ]:
OBJ_DIR = "/home/your_obj_dir"
BG_DIR = "/home/your_bg_dir"

OUT_DIR = "/home/your_output_dir"

## Actual code begins here

In [ ]:
import cv2
import csv
import numpy as np
import math
import os
import random
from PIL import Image, ImageFilter, ImageDraw, ImageChops, ImageFont
import xml.etree.ElementTree as ET
from xml.dom import minidom
from datetime import datetime

## Main Fx

In [ ]:
def generate_images():
    """
    Main function
    """
    # Output Directory
    out_dir_full = os.path.join(OUT_DIR, datetime.now().strftime("%Y_%m_%d-%H.%M.%S"))
    os.makedirs(out_dir_full)
    os.makedirs(os.path.join(out_dir_full, "images"))
    os.makedirs(os.path.join(out_dir_full, "annotations"))
    
    # Get list of files
    obj_files_list = [file for file in os.listdir(OBJ_DIR) 
         if os.path.isfile(os.path.join(OBJ_DIR, file))]
    bg_files_list = [file for file in os.listdir(BG_DIR) 
         if os.path.isfile(os.path.join(BG_DIR, file))]
    
    print(f"Processing {len(bg_files_list)} images...\n")
    # Iteratively take bgs and start pasting stuff
    for im_indx, bg in enumerate(bg_files_list):
        
        # Counter to count number of attempts spent on this image
        attempt_count = 0
        obj_count = 0
        
        # Decide number of objects to put
        obj_to_place = np.random.randint(number_objects_per_image[0], number_objects_per_image[1]+1, size=1)[0]
        
        # Open background image & Get dimensions
        image_bg = cv2.imread(os.path.join(BG_DIR, bg))
        height_bg, width_bg, _ = image_bg.shape
        img_name_noext = os.path.splitext(bg)[0]
        
        # Creates XML annotations file
        xml_annotation = create_xml_backbone(f"overlay_{img_name_noext}.jpg", os.path.join(out_dir_full, "annotations"), width_bg, height_bg)
        
        # Keep placing objects until run out of iterations or number to place is reached!
        while attempt_count < MAX_ATTEMPTS and obj_count < obj_to_place:
            
#             print("\nPlacing objects now...")
            fail_flag = True
            
            
            while fail_flag:
                
                # Choose object & rotate & magnify
                chosen_obj = random.choice(obj_files_list)
                rotate_angle = np.random.randint(rotation_range[0], rotation_range[1], size=1)[0]
                mag_factor_d = np.random.uniform(low=0, high=1, size=1)[0]
                
                # Open, rotate, and crop object
                image_obj_PIL = Image.open(os.path.join(OBJ_DIR, chosen_obj)).convert('RGB')
                image_obj_CV = rotate_and_crop(image_obj_PIL, rotate_angle, mag_factor_d)

                # Get dimensions of object
                height_obj, width_obj, _ = image_obj_CV.shape

                # Get coords to place object (x,y)
                placement_coords, try_count = determine_random_coords(image_bg, height_obj, width_obj, height_bg, width_bg)

                # Update counter
                attempt_count += try_count
                
                # Place 1 object
                if placement_coords != None:
                    obj_count += 1
                    fail_flag=False
                    # Place object onto background
#                     print("One object placed...")
                    
                    # Iterate thru pixels in object image, remove white bg
                    for x, row in enumerate(image_obj_CV):
                        for y, entry in enumerate(row):
                            if np.mean(entry)<=bg_removal_threshold:
                                image_bg[placement_coords[1]+x, placement_coords[0]+y]=image_obj_CV[x,y]
                    
                    # Update XML annotation
                    xml_annotation = update_bbox_in_XML(xml_annotation, os.path.splitext(chosen_obj)[0], placement_coords,
                       width_obj, height_obj)
                     
                if attempt_count >= MAX_ATTEMPTS:
                    break
        
            
        # Save image
        cv2.imwrite(os.path.join(out_dir_full, "images", f"overlay_{img_name_noext}.jpg"), image_bg)
        
        # Status message
        if len(bg_files_list)<30:
            print(f"{obj_count} objects placed onto {bg}. Staged image Saved.")
            if obj_count==0:
                print(f"No objects placed due to inability to find suitable spot within MAX_ATTEMPTS={MAX_ATTEMPTS}")
            print("XML Label saved.")
        elif im_indx%10==0 and im_indx!=0:
            print(f"{im_indx}/{len(bg_files_list)} images processed.")
        
        # Write and save XML
        dom = minidom.parseString(ET.tostring(xml_annotation))
        xml_out = dom.toprettyxml(indent='\t')
        with open(os.path.join(out_dir_full, "annotations", f"overlay_{img_name_noext}.xml"), "w") as f:
            f.write(xml_out)
        
    print("\nAll done!")

## Generate placement coords

In [ ]:
def determine_random_coords(image_bg, height_obj, width_obj, height_bg, width_bg):
    """
    Generate a random coordinate to place object
    Checks that it is against white background!
    
    Inputs:


    Output:
        1. Placement_coords (2x1 tuple) OR None if no suitable coords found!
           - Random coordinates to place object at (x,y)
        2. try_count
            - Number of iterations within current run
    """
    # Get coords of background
    xmin, ymin = (0,0)
    xmax, ymax = (width_bg, height_bg)

    # Generate placement coords & compute max coords and checks
        # 1. if threat image lies outside container (not allowed)
        # 2. that entire placement area is white
    pass_flag = False
    try_count = 0
    
    while not pass_flag:
        try_count += 1
        
        # Generate (x,y) coords and max_coords
        placement_coords = [np.random.randint(low=xmin, high=xmax, size=1)[0],
                            np.random.randint(low=ymin, high=ymax, size=1)[0]]
        max_coords = [placement_coords[0] + width_obj, placement_coords[1] + height_obj]        
        
#         print(max_coords)
        
        # Check if slice within image
        is_within_image = (check_within_box(xmin, ymin, xmax, ymax, placement_coords[0], placement_coords[1])
                         and check_within_box(xmin, ymin, xmax, ymax, max_coords[0], max_coords[1]))
        
#         print(f"Within img = {is_within_image}")
        
        # Check if proposed placement region is white (if required) and within image
        if is_within_image:
            if white_space_hunter:
                image_slice = image_bg[placement_coords[1]:placement_coords[1]+height_obj, 
                     placement_coords[0]:placement_coords[0]+width_obj]

                region_is_white = all_white_pixels(image_slice)
    #             print(image_slice)
            else:
                region_is_white = True
            
        else:
            region_is_white = False
        
        
        
        # Check if proposed coords passes the test
        pass_flag = (is_within_image and region_is_white) 
#         pass_flag = (is_within_image or region_is_white)
        
        # Stop trying if no suitable spot found
        if try_count >= 100:
            return None, try_count

    return placement_coords, try_count  # (x,y)

In [ ]:
def check_within_box(xmin, ymin, xmax, ymax, x, y):
    """
    Helper function: Check if coords (x,y) within bounding box
    
    True if within box
    """
    return (x > xmin and x < xmax and y > ymin and y < ymax)

## Rotate and Magnify Image

In [ ]:
def rotate_and_crop(img_PIL, rotation_deg=0, mag_factor_d=1):
    """
    Rotates image and crops it
    
    Input:
        1. img_PIL: PIL Object
    
    Output:
        1. img_cv: OpenCV Object
    """
    # Magnify
    (width, height) = (img_PIL.width, img_PIL.height)
    mag_factor = 1 + mag_factor_d * (max_magnification - 1)
    img_PIL = img_PIL.resize((int(round(width * mag_factor)), int(round(height * mag_factor))))
    
    # Rotate
    img_PIL = img_PIL.rotate(angle=rotation_deg, expand=1, fillcolor=(255,255,255))

    # Crop
    img_PIL = trim(img_PIL)
    
    # Convert to OpenCV object
    img_cv = np.array(img_PIL) 
    img_cv = img_cv[:, :, ::-1].copy()     # Convert RGB to BGR
    
    return img_cv

In [ ]:
def trim(im):
    """
    Automatically trims away the excess whitespace in a gun image to get a tighter bounding box.

    Input:
        1. im (PIL Object)
            - Background layer of im MUST be set to L=255

    Output:
        1. im_cropped (PIL Object)
           - Cropped image
    """
#     # Iterate thru pixels in object image, remove white bg
#     im_np = np.array(im)
#     for x, row in enumerate(im_np):
#         for y, entry in enumerate(row):
#             if np.mean(entry)>=bg_removal_threshold:
#                 im_np[x,y]=[255,255,255]
#     im=Image.fromarray(im_np)

    # Create bg mask of only white colour
    bg = Image.new(im.mode, im.size, 255)

    # Pixel by pixel comparison of image and background. All pixels with 255 on image should have value 0 in "diff"
    diff = ImageChops.difference(im, bg)

    # adds two images, dividing the result by scale and adding the offset
    diff = ImageChops.add(diff, diff, scale=1.0, offset=0)

    # Gets non-zero bounding box of image (ie crops image)
    bbox = diff.getbbox()

    # Crop image according to bounding box
    im_cropped = im.crop(bbox)

    return im_cropped

## Check if slice is white

In [ ]:
def all_white_pixels(image):
    """
    Returns True if all white pixels or False if not all white
    """
    white_flag = True
    
    H, W = image.shape[:2]
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    for pixel in gray:
        if np.mean(pixel)<=245:
            white_flag = False
    
    return white_flag

## For handling XML

In [ ]:
def update_bbox_in_XML(xml_annotation, selected_threat, placement_coords,
                       threat_width, threat_height):
    """
    Adds bounding box info "Object Node" into XML labels file

    Inputs:
        1. xml_annotation (XML ElementTree Object)
        2. selected_threat (Name without file extension)
        3. placement_coords (Tuple in x,y format)
        4. threat_width (x)
        5. threat_height (y)

    Returns:
        1.  xml_annotation (XML ElementTree Object)
    """
    myObject = ET.SubElement(xml_annotation, "object")

    # Get threat class
    #### UNCOMMENT THIS FOR CUSTOMISABLE THREAT CLASS, ELSE ALL SET AS "RIFLE" ######

    #   threat_tree = ET.parse(os.path.join(root, threats_dir, "annotations", f"{selected_threat}.xml"))
    #   threat_class = threat_tree.find("object").find("name").text
    
    if obj_class == "multiple":
        object_class = selected_threat.split('_')[0]
    else: 
        object_class = obj_class
    
    # Object child nodes
    ET.SubElement(myObject, "name").text = str(object_class)
    ET.SubElement(myObject, "pose").text = "Unspecified"
    ET.SubElement(myObject, "truncated").text = "0"
    ET.SubElement(myObject, "difficult").text = "0"
    bndbox = ET.SubElement(myObject, "bndbox")

    # bndbox child nodes
    ET.SubElement(bndbox, "xmin").text = str(placement_coords[0])
    ET.SubElement(bndbox, "ymin").text = str(placement_coords[1])
    ET.SubElement(bndbox, "xmax").text = str(placement_coords[0] + threat_width)
    ET.SubElement(bndbox, "ymax").text = str(placement_coords[1] + threat_height)

    return xml_annotation

In [ ]:
def create_xml_backbone(curr_filename, save_dir, img_width, img_height):
    """
    PascalVOC Format
    Creates backbone of XML label file, populating tree with everything except object bounding box labels

    Returns:
        1. xml_annotation (ElementTree Object)
    """
    # Main node
    xml_annotation = ET.Element("annotation")

    # Primary nodes
    ET.SubElement(xml_annotation, "folder").text = "images"
    ET.SubElement(xml_annotation, "filename").text = str(curr_filename)
    ET.SubElement(xml_annotation, "path").text = str(save_dir)
    source = ET.SubElement(xml_annotation, "source")
    size = ET.SubElement(xml_annotation, "size")
    ET.SubElement(xml_annotation, "segmented").text = "0"

    # Source child node
    ET.SubElement(source, "database").text = "Unknown"

    # Size child nodes
    ET.SubElement(size, "width").text = str(img_width)
    ET.SubElement(size, "height").text = str(img_height)
    ET.SubElement(size, "depth").text = "1"

    return xml_annotation